In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, PILToTensor
import time
import csv

In [2]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input)
print(target)
output = loss(input, target)
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
print(input)
print(target)
output = loss(input, target)
output.backward()

tensor([[-0.4736, -1.1207,  0.5104,  0.3454,  0.8225],
        [-0.9562, -1.0922,  1.2153, -2.3368, -0.2003],
        [-1.6257,  1.2905,  0.2290, -1.3769,  0.0810]], requires_grad=True)
tensor([0, 4, 2])
tensor([[-1.4223,  1.0200, -0.7364, -0.7279,  1.5888],
        [ 0.2138, -0.6818, -1.2128,  0.8185, -0.1387],
        [-0.8242, -0.3341,  1.9220, -0.7954, -1.7406]], requires_grad=True)
tensor([[0.1321, 0.1782, 0.0346, 0.6484, 0.0067],
        [0.3993, 0.0462, 0.1565, 0.1386, 0.2594],
        [0.4484, 0.0353, 0.1121, 0.0431, 0.3611]])


In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    # transform=PILToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6])


In [9]:
loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X.float())
        # print("pred: ",pred.shape)
        # print("y:",y.shape)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct

In [12]:
epochs = 3
for datasize in [7500,15000,30000,60000]:
    for epoch in range(epochs):
        subset_training_data = Subset(training_data, range(datasize))
        train_dataloader_subset = DataLoader(subset_training_data, batch_size=batch_size)
        print(f"Epoch {epoch+1}\n-------------------------------")
        epoch_time_start = time.perf_counter()
        train(train_dataloader_subset, model, loss_fn, optimizer)
        epoch_time_end = time.perf_counter()

        test_start_time = epoch_time_end
        accuracy = test(test_dataloader, model, loss_fn)
        test_end_time = time.perf_counter()

        epoch_duration = epoch_time_end - epoch_time_start
        test_duration = test_end_time - test_start_time

        print(f"Epoch {epoch} took {epoch_duration} seconds")
        with open("datasize_nn.csv", "a") as fp:
            wr = csv.writer(fp, dialect='excel')
            # epoch_duration, epoch, batch_size, data_size, accuracy, test_duration
            wr.writerow([epoch_duration, epoch, batch_size, datasize, accuracy, test_duration])

print("Done!")

Epoch 1
-------------------------------
loss: 2.293850  [   64/ 7500]
loss: 2.286620  [ 6464/ 7500]
Test Error: 
 Accuracy: 14.2%, Avg loss: 2.282512 

Epoch 0 took 1.0385390119627118 seconds
Epoch 2
-------------------------------
loss: 2.277496  [   64/ 7500]
loss: 2.271391  [ 6464/ 7500]
Test Error: 
 Accuracy: 20.8%, Avg loss: 2.264575 

Epoch 1 took 1.0930766009842046 seconds
Epoch 3
-------------------------------
loss: 2.261476  [   64/ 7500]
loss: 2.256266  [ 6464/ 7500]
Test Error: 
 Accuracy: 26.0%, Avg loss: 2.246580 

Epoch 2 took 1.153512449003756 seconds
Epoch 1
-------------------------------
loss: 2.245348  [   64/15000]
loss: 2.240552  [ 6464/15000]
loss: 2.200639  [12864/15000]
Test Error: 
 Accuracy: 38.6%, Avg loss: 2.208948 

Epoch 0 took 2.146108693035785 seconds
Epoch 2
-------------------------------
loss: 2.210851  [   64/15000]
loss: 2.206259  [ 6464/15000]
loss: 2.155030  [12864/15000]
Test Error: 
 Accuracy: 43.8%, Avg loss: 2.166734 

Epoch 1 took 2.0787526

In [13]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [14]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>